In [14]:


# {year} {series ID} {Race ID}
year = 2025
series_id = 1
race_id = "ABC456"

In [74]:
import warnings
import requests
import pandas as pd
from src.codes import FLAG_CODE

# endpoint for race list
#https://cf.nascar.com/cacher/2023/race_list_basic.json

class Schedule:
    
    def __init__(self, year, series_id):
        self.year = year
        self.series_id = series_id
        self.races = []
        self.data = pd.DataFrame()
        self.fetch_races()

    def fetch_races(self):
        """Fetch the race list for the specified year and series ID."""
        url = f"https://cf.nascar.com/cacher/{self.year}/race_list_basic.json"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            # Filter the races by series ID
            race_list = data[f'series_{self.series_id}']
            self.races = [race for race in race_list if race['series_id'] == self.series_id]
            self.data = pd.DataFrame(self.races)
            
        else:
            warnings.warn(f"Failed to fetch race list: {response.status_code}")
    def get_completed_races(self):
        """Return a list of completed races."""
        return self.data[self.data['winner_driver_id'].notna()]['race_name'].tolist()


schedule = Schedule(year, series_id)
schedule.data.head()

,race_id,series_id,race_season,race_name,race_type_id,restrictor_plate,track_id,track_name,date_scheduled,race_date,...,satellite_radio_broadcaster,master_race_id,inspection_complete,playoff_round,is_qualifying_race,qualifying_race_no,qualifying_race_id,has_qualifying,winner_driver_id,pole_winner_laptime
0,5543,1,2025,Cook Out Clash at Bowman Gray,2,False,159,Bowman Gray Stadium,2025-02-02T20:00:00,2025-02-02T20:00:00,...,SIRIUSXM,3222,True,0,False,0,-1,False,4062.0,None
1,5544,1,2025,Duel 1 at DAYTONA,2,True,105,Daytona International Speedway,2025-02-13T19:00:00,2025-02-13T19:00:00,...,SIRIUSXM,3217,True,0,False,0,-1,False,4025.0,None
2,5545,1,2025,Duel 2 at DAYTONA,2,True,105,Daytona International Speedway,2025-02-13T20:45:00,2025-02-13T20:45:00,...,SIRIUSXM,3218,True,0,False,0,-1,False,4180.0,None
3,5546,1,2025,DAYTONA 500,1,True,105,Daytona International Speedway,2025-02-16T14:30:00,2025-02-16T14:30:00,...,SIRIUSXM,385,True,0,False,0,-1,False,4184.0,None
4,5547,1,2025,Ambetter Health 400,1,True,111,Atlanta Motor Speedway,2025-02-23T15:00:00,2025-02-23T15:00:00,...,SIRIUSXM,3335,True,0,False,0,-1,False,4153.0,None


In [ ]:
#https://cf.nascar.com/cacher/2023/2/5314/lap-times.json
#

class Race:
    def __init__(self, year, series_id,race_id=None,live=False):
        # Initialize race metadata
        self.year = year
        self.live = live
        self.series_id = series_id
        self.race_id = race_id

        # Race Data
        self.name = None
        self.laps = None
        self.pit_stops = None
        self.events = None
        self.start_time = None
        self.end_time = None
        self.winner = None
        self.data = {}
        self.fetch_laps()
        self.get_pit_stops()
        self.fetch_events()
        
    def fetch_laps(self):
        """Fetch lap times for the specified race ID."""
        if self.live:
            url = f"https://cf.nascar.com/cacher/live/series_{self.series_id}/{self.race_id}/lap-times.json"
        else:
            url = f"https://cf.nascar.com/cacher/{self.year}/{self.series_id}/{self.race_id}/lap-times.json"

        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            self.data = data
            lap_times = []
            
            for i in data['laps']:
                driver = i.get('FullName')
                number = i.get('Number')
                manufacturer = i.get('Manufacturer')
                for j in i.get('Laps', []):
                    lap_times.append({
                        'Driver': driver,
                        'Number': number,
                        'Manufacturer': manufacturer,
                        'Lap': j.get('Lap'),
                        'lap_time': j.get('LapTime'),
                        'lap_speed': j.get('LapSpeed'),
                        'position': j.get('RunningPos'),
                    })
            self.laps = pd.DataFrame(lap_times)
            self.laps['Lap'] = self.laps['Lap'].astype(int)
            self.laps['lap_time'] = pd.to_timedelta(self.laps['lap_time'])
            self.laps['lap_speed'] = self.laps['lap_speed'].astype(float)
        else:
            print(f"Failed to retrieve lap data: {response.status_code}")
            print(f"url: {url}")

    def get_pit_stops(self):
        """Get pit stop information for the race."""
        if self.live:
            url = f"https://cf.nascar.com/cacher/live/series_{self.series_id}/{self.race_id}/live-pit-data.json"
        else:
            url = f"https://cf.nascar.com/cacher/{self.year}/{self.series_id}/{self.race_id}/live-pit-data.json"

        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            stops = []
            for i in data:
                stops.append({
                    'Driver': i.get('driver_name'),
                    'Lap': i.get('lap_count'),
                    'Manufacturer': i.get('vehicle_manufacturer'),
                    'pit_in_flag_status': i.get('pit_in_flag_status'),
                    'pit_out_flag_status': i.get('pit_out_flag_status'),
                    'pit_in_race_time': i.get('pit_in_race_time'),
                    'pit_out_race_time': i.get('pit_out_race_time'),
                    'total_duration': i.get('total_duration'),
                    'box_stop_race_time': i.get('box_stop_race_time'),
                    'box_leave_race_time': i.get('box_leave_race_time'),
                    'pit_stop_duration': i.get('pit_stop_duration'),
                    'in_travel_duration': i.get('in_travel_duration'),
                    'out_travel_duration': i.get('out_travel_duration'),
                    'pit_stop_type': i.get('pit_stop_type'),
                    'left_front_tire_changed': i.get('left_front_tire_changed'),
                    'left_rear_tire_changed': i.get('left_rear_tire_changed'),
                    'right_front_tire_changed': i.get('right_front_tire_changed'),
                    'right_rear_tire_changed': i.get('right_rear_tire_changed'),
                    'previous_lap_time': i.get('previous_lap_time'),
                    'next_lap_time': i.get('next_lap_time'),
                    'pit_in_rank': i.get('pit_in_rank'),
                    'pit_out_rank': i.get('pit_out_rank'),
                    'positions_gained_lost': i.get('positions_gained_lost'),
                })
            # store both list and a DataFrame for convenience
            self.pit_stops = pd.DataFrame(stops) if stops else pd.DataFrame()
        else:
            print(f"Failed to retrieve pit stop data: {response.status_code}")
            print(f"url: {url}")

    def fetch_events(self):
        """ Fetch lap events and flags"""
        if self.live:
            url = f"https://cf.nascar.com/cacher/live/series_{self.series_id}/{self.race_id}/lap-notes.json"
        else:
            url = f"https://cf.nascar.com/cacher/{self.year}/{self.series_id}/{self.race_id}/lap-notes.json"

        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            events = []
            laps = data.get('laps')
            print(laps)
            
            for k,v in laps.items():
                for j in v:
                    events.append({
                        'Lap': k,
                        'Flag_State': j.get('FlagState'),
                        'Flag': None,
                        'note': j.get('Note'),
                        #'note_id': j.get('NoteID'),
                        'driver_ids': j.get('DriverIDs'),
                    })
            self.events = pd.DataFrame(events) if events else pd.DataFrame()
            if not self.events.empty:
                self.events['Flag'] = self.events['Flag_State'].map(FLAG_CODE)

        else:
            print(f"Failed to retrieve lap events: {response.status_code}")
            print(f"url: {url}")

race = Race(year, series_id, 5546)
race2 = Race(2023,2,5314)


In [80]:
race.fetch_events()
race.events

{'0': [{'FlagState': 8, 'Note': 'To the rear:  #5, #6, #7, #35, #48, #54, #88, #99 (backup cars), #91 (engine change), #47 (inspection failures', 'NoteID': 55702, 'DriverIDs': [4030, 1816, 4172, 4269, 4045, 4368, 4469, 4113, 4405, 3888]}], '1': [{'FlagState': 1, 'Note': '#19 leads the field to the green on the inside, #2 outside.  #19 gets an initial push and holds on to the lead.', 'NoteID': 55703, 'DriverIDs': [4228, 4180, 4228]}], '3': [{'FlagState': 1, 'Note': '#19, #23 and #2 get single file in front of the field', 'NoteID': 55704, 'DriverIDs': [4228, 4025, 4180]}], '5': [{'FlagState': 1, 'Note': '#77 reports fuel pressure issues and loses the draft.', 'NoteID': 55705, 'DriverIDs': [4326]}], '6': [{'FlagState': 1, 'Note': '#24 gets to the front of the outside line, with the #10 battling on the inside.', 'NoteID': 55706, 'DriverIDs': [4184, 4013]}], '7': [{'FlagState': 1, 'Note': 'A top lane led by #2 and #22 form and begin to move forward.', 'NoteID': 55707, 'DriverIDs': [4180, 38

,Lap,Flag_State,Flag,note,driver_ids
0,0,8,Warm Up,"To the rear: #5, #6, #7, #35, #48, #54, #88, ...","[4030, 1816, 4172, 4269, 4045, 4368, 4469, 411..."
1,1,1,Green,#19 leads the field to the green on the inside...,"[4228, 4180, 4228]"
2,3,1,Green,"#19, #23 and #2 get single file in front of th...","[4228, 4025, 4180]"
3,5,1,Green,#77 reports fuel pressure issues and loses the...,[4326]
4,6,1,Green,"#24 gets to the front of the outside line, wit...","[4184, 4013]"
...,...,...,...,...,...
80,194,1,Green,"Top 3 of #2, #11 and #20 go single file until ...","[4180, 1361, 4153, 1361, 4180, 1361, 4153, 4180]"
81,195,1,Green,#20 makes a save after #41 gives a push in Tur...,"[4153, 4104, 4153, 1361]"
82,196,2,Yellow,Caution: #20 spins after a push from the #41 ...,"[4153, 4104, 4070, 4059, 4059, 1816, 4113, 403..."
83,200,1,Green,"Restart: #11 leads from the inside, #2 outsid...","[1361, 4180, 1361, 4180, 4104]"


In [59]:
race.laps

,Driver,Number,Manufacturer,Lap,lap_time,lap_speed,position
0,William Byron,24,Chv,0,NaT,NaN,5
1,William Byron,24,Chv,1,0 days 00:00:00.000000052,172.629,5
2,William Byron,24,Chv,2,0 days 00:00:00.000000047,188.045,5
3,William Byron,24,Chv,3,0 days 00:00:00.000000047,187.633,4
4,William Byron,24,Chv,4,0 days 00:00:00.000000047,187.668,2
...,...,...,...,...,...,...,...
7474,AJ Allmendinger,16,Chv,38,0 days 00:00:00.000000047,189.498,12
7475,AJ Allmendinger,16,Chv,39,0 days 00:00:00.000000047,189.139,15
7476,AJ Allmendinger,16,Chv,40,0 days 00:00:00.000000047,189.406,14
7477,AJ Allmendinger,16,Chv,41,0 days 00:00:00.000000047,189.450,14


In [7]:
laps = race.test['laps']
lap_times = []

for i in laps:
    driver = i['FullName']
    number = i['Number']
    manufacturer = i['Manufacturer']
    for j in i['Laps']:
        lap_times = lap_times
        lap_times.append({
            'Driver': driver,
            'Number': number,
            'Manufacturer': manufacturer,
            'Lap': j['Lap'],
            'lap_time': j['LapTime'],
            'lap_speed': j['LapSpeed'],
            'position': j['RunningPos'],
        })
        
lap_times = pd.DataFrame(lap_times)
lap_times

TypeError: 'NoneType' object is not subscriptable

In [18]:
race.laps.Driver.unique()

AttributeError: 'NoneType' object has no attribute 'Driver'